# Auslese und Darstellung von KOSTRA-Werten 
(DWD KOSTRA2010R, Stand 01/2021)

Verwendete python-Module: numpy, matplotlib, ipyleaflet, ipywidgets, json, pandas, plotly


## 1. Kartenanwendung: Auswahl einer KOSTRA-Zelle

In [1]:
#Kartenanwendung
import numpy as np
import json
import numpy as np
from ipywidgets import widgets, IntSlider, jslink, HTML, Layout,Label,VBox
from ipyleaflet import Map, GeoData,GeoJSON, basemaps, LayersControl,Popup, ScaleControl, FullScreenControl, WidgetControl,WMSLayer
from IPython.display import Javascript, display

Tlist=[1.001,2,3,5,10,20,30,50,100] #Jährlichkeiten
Dlist=[5,10, 15, 20, 30, 45, 60, 90, 120, 180, 240, 360, 540,720, 1080, 1440, 2880, 4320] #Dauerstufen [min]
values=np.load('KOSTRA2010R_Matrix_numpy.npy')#KOSTRA-Werte (Numpy-Array) laden
cell_ids=values[0,:,0]


# Map and label widgets
map = Map(center=[51, 8], zoom=8)
label = Label(layout=Layout(width='100%'))
label.value= str(55010) #dummy Wert bis zur ersten Selektion (Zelle Kölner Dom)
zoom_slider = IntSlider(description='Zoom level:', min=0, max=15, value=8)
jslink((zoom_slider, 'value'), (map, 'zoom'))
widgetControl = WidgetControl(widget=zoom_slider, position='topright')
map.add_control(widgetControl)
map.add_control(LayersControl())

wms_dtk = WMSLayer(
    name='NW_DTK_Sammeldienst_sw',
    url='https://www.wms.nrw.de/geobasis/wms_nw_dtk?',
    layers='nw_dtk_sw',
    format='image/png',
    transparent=True,
    attribution='nw_dtk_sw © geobasis nrw'
)


map.add_layer(wms_dtk)
map.add_control(ScaleControl(position='bottomleft'))
map.add_control(FullScreenControl(position='topright'))


# geojson layer with hover handler
with open('kostra.json') as f:
    data = json.load(f)
    

for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'blue',
        'fillOpacity': 0.05
    }

layer = GeoJSON(name="DWD-KOSTRA2010R-Zellen",data=data, hover_style={'fillColor': 'red'})
map.add_layer(layer)

#def hover_handler(event=None, feature=None, id=None, properties=None):
#    label.value = str(properties['INDEX_RC'])
#layer.on_hover(hover_handler)
#map.add_layer(layer)
#ipyw.VBox([map, label])

def click_handler(type=None,event=None,feature=None,properties=None,id=None):
    label.value= str(properties['INDEX_RC'])    
    coords=(feature['geometry']['coordinates'][0])
    #print(np.array(coords))
    lo=(np.mean(coords,axis=0)[0])
    la=(np.mean(coords,axis=0)[1])
    text=str(feature['properties']['INDEX_RC'])
    #print(text)
    message = HTML(value=str('INDEX_RC: '+text))
    popup = Popup(location=(la, lo), child=message, close_button=True, auto_close=False, close_on_escape_key=False)
    map.add_layer(popup)
    #run_all_cells_below()
    
layer.on_click(click_handler)
cell_id=int(label.value)
#VBox([map, label])
map

Map(center=[51, 8], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## 2. Auswertung der selektierten KOSTRA-Zelle

In [2]:

import plotly.graph_objects as go
import pandas as pd
import numpy as np
from IPython.display import clear_output
button = widgets.Button(description="Diagramme aktualisieren!",layout=Layout(width='50%', height='50px'))
output = widgets.Output()
display(widgets.VBox([button,output]))
#display(button)


def on_button_clicked(b):
    #with output:
    output.clear_output()
    #clear_output()
    #print("neuer Plot wird für Zelle " + str(label.value)+ " erstellt.")
    setup_ui()
    #display(widgets.VBox([button,a]))

def setup_ui():
    with output:
        a=plot_regen_x3()
        display(a)
    return output

def plot_regen_x3():
    cell_id=int(label.value) 
    cell_row=np.where(cell_id==cell_ids)[0]
    vals=values[:,cell_row,:]
    #Plot 3D-Surface
    X=Dlist
    Y=Tlist
    X, Y = np.meshgrid(X, Y)
    Z=vals[:,0,1:]
    Z=np.swapaxes(Z,1,0)
    fig = go.FigureWidget(data=[go.Surface(z=Z, x=X, y=Y)])
    fig.update_layout(title='Regenhöhenoberfläche', autosize=False)
    fig.update_layout(scene = dict(
                        xaxis_title='Dauerstufe [min]',
                        yaxis_title='Jährlichkeit [a]',
                        zaxis_title='h [mm]'),
                        width=600,height=600,
                        margin=dict(r=20, b=10, l=10, t=10))

    camera = dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=-1.9, y=-1.6, z=1.25)
    )

    fig.update_layout(scene_camera=camera)

    #2D-Plot 1
    import plotly.express as px
    #cell_id=int(label.value)
    #cell_row=np.where(cell_id==cell_ids)[0]
    #vals=values[:,cell_row,:]
    tab=vals.transpose(0,2,1).reshape(-1,vals.shape[2])
    df=pd.DataFrame(tab)
    df.columns = ["INDEX_RC"]+['Tn_'+str(round(i)) for i in Tlist]
    df['D_min']=Dlist
    df.iloc[:3,1::]
    df_=df.T
    fig1 = px.line(df, x="D_min", y=['Tn_'+str(round(i)) for i in Tlist],log_x=True)
    fig1.update_traces(mode="markers+lines", hovertemplate=None)
    fig1.update_layout(hovermode="x",legend_traceorder='reversed')
    fig1.update_layout(title="DWD-KOSTRA2010R Regenhöhenlinien h [mm] über D [min],KOSTRA-Zelle "+str(cell_id),
                        xaxis_title="Dauerstufe [min]",
                        yaxis_title="h [mm]",
                        legend_title="Jährlichkeit [a]",
                         width=900,
                         height=500)
    figw1=go.FigureWidget(fig1)

    #2D-PLot 2
    df_=df.T.iloc[1:-1]
    df_.columns = ['D_'+str(round(i)) for i in Dlist]
    df_['Tn']=Tlist
    fig2 = px.line(df_, x="Tn", y=['D_'+str(round(i)) for i in Dlist],log_x=True)
    fig2.update_traces(mode="markers+lines", hovertemplate=None)
    fig2.update_layout(hovermode="x",legend_traceorder='reversed')
    fig2.update_layout(title="DWD-KOSTRA2010R Regenhöhenlinien h [mm] über Tn [a], KOSTRA-Zelle "+str(cell_id),
                        xaxis_title="Jährlichkeit[a]",
                        yaxis_title="h [mm]",
                        legend_title="Duerstufe [min]",
                         width=900,
                         height=500)
    figw2=go.FigureWidget(fig2)


    fig_subplots=VBox([fig,figw2,figw1])
    #display(fig_subplots)
    return fig_subplots

setup_ui()
button.on_click(on_button_clicked)

## CSV-Download der selektierten KOSTRA-Tabelle

In [9]:

from ipywidgets import HTML
from IPython.display import display
import base64

button1 = widgets.Button(description="CSV-Datei erzeugen",layout=Layout(width='50%', height='50px'))
output1 = widgets.Output()
display(widgets.VBox([button1,output1]))
#display(button1)


def on_button1_clicked(b):
    #with output:
    output1.clear_output()
    #clear_output()
    #print("neuer Plot wird für Zelle " + str(label.value)+ " erstellt.")
    get_csv()
    #df_
    #display(widgets.VBox([button,a]))

@output1.capture()
def get_csv():
    cell_id=int(label.value)
    cell_row=np.where(cell_id==cell_ids)[0]
    vals=values[:,cell_row,:]
    tab=vals.transpose(0,2,1).reshape(-1,vals.shape[2])
    df=pd.DataFrame(tab)
    df.columns = ["INDEX_RC"]+['Tn_'+str(round(i)) for i in Tlist]
    df['D_min']=Dlist
    df.iloc[:3,1::]
    df_=df.T
    df_=df.T.iloc[1:-1]
    df_.columns = ['D_'+str(round(i)) for i in Dlist]
    df_['Tn']=Tlist
    res = str(df_)
    #FILE
    filename = 'KOSTRA2010R_Zelle'+str(cell_id)+'.txt'
    b64 = base64.b64encode(res.encode())
    payload = b64.decode()

    #BUTTONS
    html_buttons = '''<html>
    <head>
    <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <a download="{filename}" href="data:text/csv;base64,{payload}" download>
    <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">CSV-File Download</button>
    </a>
    </body>
    </html>
    '''

    html_button = html_buttons.format(payload=payload,filename=filename)
    display(HTML(html_button))
    #display(widgets.VBox([HTML(html_button),print(df_)]))

button1.on_click(on_button1_clicked)

Button(description='CSV-Datei erzeugen', layout=Layout(height='50px', width='50%'), style=ButtonStyle())